In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.connect("../../data/survey.db")

cursor = connection.cursor()
cursor.execute("SELECT Site.lat, Site.long FROM Site;")
results = cursor.fetchall()
for result in results:
    print(result)
cursor.close()
connection.close()

In [ ]:
def get_name(connection, person_identifier):
    query = f"SELECT personal || ' ' || family FROM Person WHERE id='{person_identifier}';"
    cursor = connection.cursor()
    cursor.execute(query)
    (name,), *_ = cursor.fetchall()
    cursor.close()
    return name


In [ ]:
connection = sqlite3.connect("../../data/survey.db")
name = get_name(connection, "dyer")
connection.close()

In [ ]:
name

In [ ]:
def get_name(connection, person_identifier):
    query = "SELECT personal || ' ' || family FROM Person WHERE id=?;"
    cursor = connection.cursor()
    cursor.execute(query, [person_identifier])
    (name,), *_ = cursor.fetchall()
    cursor.close()
    return name


In [ ]:
connection = sqlite3.connect("../../data/survey.db")
name = get_name(connection, "dyer")
connection.close()

In [ ]:
name

## Filling a Table vs. Printing Values

In [ ]:
import numpy as np

prng = np.random.RandomState(42)
numbers = prng.uniform(low=10.0, high=25.0, size=1000000)

In [ ]:
%%timeit

connection = sqlite3.connect("../../data/original.db")
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS Pressure;")
cursor.execute("CREATE TABLE Pressure (reading float not null);")

query = "INSERT INTO Pressure values (?);"
for number in numbers:
    cursor.execute(query, [number])

cursor.close()
connection.commit()
connection.close

In [ ]:
%%timeit

with open("../../data/original.txt", 'w') as out:
    for number in numbers:
        out.write(f"{number}\n")

## Filtering in SQL vs. Filtering in Python

In [ ]:
%%timeit

original_connection = sqlite3.connect("../../data/original.db")
original_cursor = original_connection.cursor()
original_cursor.execute("SELECT * FROM Pressure;")
results = original_cursor.fetchall()
original_cursor.close()
original_connection.close()

backup_connection = sqlite3.connect("../../data/backup.db")
backup_cursor = backup_connection.cursor()
backup_cursor.execute("DROP TABLE IF EXISTS Pressure;")
backup_cursor.execute("CREATE TABLE Pressure (reading float not null);")

query = "INSERT INTO Pressure values (?);"
for (number,) in results:
    if number > 20.0:
        backup_cursor.execute(query, [number])
    else:
        pass
backup_cursor.close()
backup_connection.commit()
backup_connection.close()

In [ ]:
%%timeit

original_connection = sqlite3.connect("../../data/original.db")
original_cursor = original_connection.cursor()
original_cursor.execute("SELECT * FROM Pressure WHERE reading > 20.0;")
results = original_cursor.fetchall()
original_cursor.close()
original_connection.close()

backup_connection = sqlite3.connect("../../data/backup.db")
backup_cursor = backup_connection.cursor()
backup_cursor.execute("DROP TABLE IF EXISTS Pressure;")
backup_cursor.execute("CREATE TABLE Pressure (reading float not null);")

query = "INSERT INTO Pressure values (?);"
for (number,) in results:
    backup_cursor.execute(query, [number])
    
backup_cursor.close()
backup_connection.commit()
backup_connection.close()